# Compression 2

We finish the work started in the previous chapter: we present the tools used in JPEG and also in JPEG2000.


## Setting

### Pull data

In [ ]:
"we import (=clone) all the data or just update (=pull) them"

import os

if not os.path.exists("assets_signal"):
    print("the directory assets_signal is create")
    !git clone https://github.com/vincentvigon/assets_signal
else:
    print("the directory assets_signal is updated")
    %cd assets_signal
    !git pull https://github.com/vincentvigon/assets_signal
    %cd ..

### Import

In [ ]:
%reset -f

In [ ]:
import numpy as np
np.set_printoptions(linewidth=50000,precision=1,suppress=True,)

import scipy
import scipy.signal
import scipy.fftpack

import  matplotlib.pyplot as plt
plt.style.use("default")

import imageio
import IPython
import json

## YCbCr color encoding

In the previous chapter, we saw the JPEG compression for gray-level images. What about color images?

### Principle

In [ ]:
IPython.display.Image("assets_signal/YCrCb.jpg")



YCbCr is the color encoding used in the JPEG compression. The 3 components are Y (luma), Cb ( Chroma blue) and Cr (Chroma red). They are linear transformations of the R,G,B components.

* Y  represent the gray level of the image:

        Y = 0.299*R +  0.587*G + 0.114*B

The 3 constants which appear in this formula are the same 3 constants used usualy to transform a color image into a gray-level image.

*  (Cb, Cr)  are sort of projections onto the plane which is orthogonal to the Y-axis: see drawing above.



One can compress more the components Cb and Cr than the component Y:  because  human eyes are more sensible to the lumunosity than to the hue.


Remark: A similar compression could be done with HSV of HSL encoding (using V or L in place of Y), but formulas for the  YCbCr encoding are linear and consequently faster to compute.




In [ ]:
def rgb2ycbcr(rgb):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = rgb @ xform.T
    ycbcr[:,:,[1,2]] += 128.
    return np.clip(ycbcr,0,255).astype(np.uint8)

def ycbcr2rgb(ycbcr):
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    copy=ycbcr.astype(np.float64)
    copy[:,:,[1,2]] -= 128.
    rgb=copy@xform.T
    return np.clip(rgb,0,255).astype(np.uint8)

***To you:***

* $(2\heartsuit)$ From the code above, write the formulas of the change of variable  $(R,G,B)\leftrightarrow (Y,C_b,C_r)$.

* $(1\heartsuit)$  Is it precisely a linear transform?

* $(1\heartsuit)$   Show that, without the `np.clip()`, some $(Y,C_b,C_r)$ triple of `unint8` does not give a proper RGB color.



In [ ]:
img = imageio.imread("assets_signal/babouin/babouin_moyen.jpg")
img.shape

In [ ]:
fig,axs=plt.subplots(1,3)
for i in [0,1,2]:
    axs[i].imshow(img[:,:,i],cmap="gray")
    axs[i].axis("off")

In [ ]:
img_YCbCr= rgb2ycbcr(img)
img_YCbCr.shape

In [ ]:
fig,axs=plt.subplots(1,3)
for i in [0,1,2]:
    axs[i].imshow(img_YCbCr[:,:,i],cmap="gray")
    axs[i].axis("off")

In [ ]:
img_back=ycbcr2rgb(img_YCbCr)
plt.imshow(img_back);

***To you:*** $(2\heartsuit)$ Transform the image by setting to 0 the component `Cb`, and then the component `Cr`, then the component `Y`.

### Compress color

Here is the recipe of the JPEG pipeline for color images:

* Take a RGB color.
* Change it to YCbCr.
* Keep the Y component unchanged.
* For Cb and Cr make a sub-sampling: for each $n\times n$ square of pixels, you keep only the mean.
* Then make the discrete-cosinus compression of the 3 channels as explained in the previous chapter.


***To you:***  In this exercice, you do not have to do the last step of the JPEG compression (no  cosinus-transform).


* $(8 \heartsuit)$ Make a function which  compress the babouin for $n=2,4,8,16,32$ (actualy, in JPEG $n=2$). Help: use a rolling window (and not a convolution which would compute too much means). Make a function which decompress.  Test the process compression+decompression for various $n$.  
* $(2\heartsuit)$ Compute theoriticaly the compression rate according to $n$.
* $(5 \heartsuit\flat)$ generalize your programs so that it works for any image-size and any $n$.   Help: use the fonction `np.pad` with a good `mode` argument (see docstring).






###  JPEG  Color

***To you:*** $(8\heartsuit\flat)$. Make a function that perform the JPEG compression for color images. The result must be a string as in the previous chapter. Compare the length of this string with the lengt of the image simply converted to a string.

For the quantisation, use the two following matrices.

***Remark:*** To do the previous exercice, you probably copy-paste your grey-level JPEG program in the present notebook. It is not a very good habit: the good technic  to `import` functions from a file to an other (we will see how to do this later on).


In [ ]:
Qlum_str="""
16 11 10 16 24 40 51 61
12 12 14 19 26 58 60 55
14 13 16 24 40 57 69 56
14 17 22 29 51 87 80 62
18 22 37 56 68 109 103 77
24 36 55 64 81 104 113 92
49 64 78 87 103 121 120 101
72 92 95 98 112 100 103 99
"""

Qchrom_str="""
17 18 24 47 99 99 99 99
18 21 26 66 99 99 99 99
24 26 56 99 99 99 99 99
47 66 99 99 99 99 99 99
99 99 99 99 99 99 99 99
99 99 99 99 99 99 99 99
99 99 99 99 99 99 99 99
99 99 99 99 99 99 99 99
"""

def to_matrix(Q_str):
    Q=Q_str.split()
    Q=np.array(Q,dtype=np.int64)
    return Q.reshape([8,8])

Qlum=to_matrix(Qlum_str)
print(Qlum)
print()
Qchrom=to_matrix(Qchrom_str)
print(Qchrom)

## Huffmann encoding

 ### Principle

 We have letters to encode with sequences of 0 and 1. This letters  appears with some given frequencies: ex:

    letter_to_freq={"A":15,"B":7,"C":6,"D":6,"E":5}

 To gain place, we want to attribute shortest sequences to highest frequecies. Ex:
    
    encoding={'A': '1', 'B': '011', 'C': '010', 'D': '001', 'E': '000'}
    
The constraint is that no encoding-sequence is the prefix of an other encoding-sequence. This constraint is satisfied with the above encoding.
    
    
***To you:*** $(2\heartsuit)$ Why this constraint? Help:  If you have no idea, try to code and decode some word.  


The constraint can be express graphicaly: all letters can be placed at the leaf of a binary tree whose branchs are labeled with 0 and 1. The encoding-sequence is given by the path from the root to the leaf.

Above is the construction of a tree corresponding to our `letter_to_freq`. This tree is obtained by  progressive gathering: this is the principle of the 'Huffman encoding'.

In [ ]:
IPython.display.Image("assets_signal/huffman.png")

***To you:*** $(1\heartsuit)$ How do we choose the order of the consecutive gatherings?

### Code

Here is a program (made by a student of Strasbourg) which create the Huffman encoding for a given alphabet with frequencies.

In [ ]:
def codageHuffman(dico:dict):
    dact=dico.copy()
    L=[]
    totOcurrence=0
    for key,values in dico.items():
        totOcurrence+=values
        L+=[([key],values)]
    while len(L)!=1:
        min1=totOcurrence
        min2=totOcurrence
        for numKey,occurence in enumerate(L):
            if occurence[1]<min1 or occurence[1]<min2:
                if min1<=min2:
                    min2=occurence[1]
                    indiceMin2=numKey
                else:
                    min1 = occurence[1]
                    indiceMin1 = numKey

        if min1<min2 : indiceMin2,indiceMin1=indiceMin2,indiceMin1

        if len(L[indiceMin1][0])==1:
            dact[L[indiceMin1][0][0]]='0'
        else:
            for i in range(len(L[indiceMin1][0])):
                    dact[L[indiceMin1][0][i]]+='0'

        if len(L[indiceMin2][0])==1:
            dact[L[indiceMin2][0][0]]='1'
        else:
            for i in range(len(L[indiceMin2][0])):
                dact[L[indiceMin2][0][i]]+='1'


        L.append((L[indiceMin2][0]+L[indiceMin1][0],min1+min2))
        del L[max(indiceMin1,indiceMin2)]
        del L[min(indiceMin1,indiceMin2)]


    for key in dact.keys():
        a = ''
        for let in range(len(dact.get(key))):
            a+=dact.get(key)[len(dact.get(key))-let-1]
        dact[key]=a
    return dact

In [ ]:
dico={'a':200,'b':7,'c':9,'d':10,'e':10,'f':100}
print(codageHuffman(dico))

***To you:***

* $(1\heartsuit)$ On a sheet of paper, draw the consecutive trees which lead to this encoding.
* $(1\heartsuit)$ Does this program gives THE good result?

***To you:*** $(3\heartsuit)$ Associate the three following code the the good dictionary. Try to do this before to runing a program.

    {'a': '01', 'b': '000', 'c': '001', 'd': '1'}
    {'a': '1', 'b': '010', 'c': '011', 'd': '00'}
    {'a': '001', 'b': '01', 'c': '000', 'd': '1'}

In [ ]:
dico1={'a':200,'b':50,'c':3,'d':400}
dico2={'a':400,'b':50,'c':3,'d':400}
dico3={'a':40,'b':250,'c':3,'d':400}

### Letter frequencies in french and english


Below are the frequencies of the letter for french and english.

***To you:***

* $(3\heartsuit)$ Make some graphical comparison between the two languages. Look for letters that are 5 times more frequent in english than in french.  
* $(3\heartsuit)$  Create encodings for each language. Help:
    * start to cut strings with  `split("\n")`.
    * Use `float()` to convert string into floats.
    * You can extract subtring with the slicing. ex: `hello[1:3]` gives `el`.
* $(3\heartsuit)$   Compute the proportion of place you win with these encodings. Help: you have to compute fistly the length of an encoding with constant length (like the ASCII encoding). Then the length with the Huffman encoding (taking in concideration the frequencies).



In [ ]:

english_str="""
a	0.08167
b	0.01492
c	0.02782
d	0.04253
e	0.12702
f	0.02228
g	0.02015
h	0.06094
i	0.06966
j	0.00153
k	0.00772
l	0.04025
m	0.02406
n	0.06749
o	0.07507
p	0.01929
q	0.00095
r	0.05987
s	0.06327
t	0.09056
u	0.02758
v	0.00978
w	0.02360
x	0.00150
y	0.01974
z	0.00074
"""

french_str="""
A	8,13%
B	0,93%
C	3,15%
D	3,55%
E	15,10%
F	0,96%
G	0,97%
H	1,08%
I	6,94%
J	0,71%
K	0,16%
L	5,68%
M	3,23%
N	6,42%
O	5,27%
P	3,03%
Q	0,89%
R	6,43%
S	7,91%
T	7,11%
U	6,05%
V	1,83%
W	0,04%
X	0,42%
Y	0,19%
Z	0,21%
Œ	0,01%
À	0,54%
Â	0,03%
Ç	0,00%
È	0,35%
É	2,13%
Ê	0,24%
Ë	0,01%
Î	0,03%
Ï	0,00%
Ô	0,07%
Ù	0,02%
Û	0,05%
Ü	0,02%
"""

### Entropy







Consider $p$ the vector of frequencies of letters, renormalized to get probabilities. Ex:


In [ ]:
letter_to_freq={"A":15,"B":7,"C":6,"D":6,"E":5}
l=len(letter_to_freq)
probas=np.empty([l])
for i,val in enumerate(letter_to_freq.values()):
    probas[i]=val

probas/=probas.sum()
probas

The associated entropy is
$$
H(p) = - \sum p_i  \log_2( {p_i})
$$
where $\log_2$ is the logarithm in basis 2: $\log_2(x)= \log(x)/\log(2)$. And we use the convention $0\log(0)=0$ when a proba is zero.


Considere $f$ a binary encoling i.e an application which associate to any letter $a$ the conding sequence $f(a_i)$. The Average length of $f$ is
$$
AL_p(f) = \sum_i  lenght(f(a_i))\ p_i
$$
where $p_i$ is the probability of the letter $a_i$.

***Remark:*** To reply to the exercice of the previous section, you already have computed some average lengths.


The shanon information theorem says that, for any encoding, we have:
$$
H(p) \leq AL_p(f)
$$
And in particular, if $f$ is the huffman encoding, we get:
$$
H(p) \leq AL_p(f)  \leq H(p) + 1
$$


*** To you:***

* $(2\heartsuit)$ Check the previous inequalities for your french and english encoding.
* $(2\heartsuit)$ What must we change in these formulas if we work with hexadecimal encoding instead of binary encoding?


###  JPEG and encodings





In [ ]:
IPython.display.Image("assets_signal/JPEG_process.png",width=600)

We call "entropy coding" a code that associate to the most frequent symbols the shortest code. The final step of JPEG is to encode the quantized coefficients with an entropy coding. The JPEG standard let you two choices:

* The Huffman coding which we present before.
* The arithmetic coding which gives better result (about 5% shorter) but require more calculus, and so, it is rarely used.


But just before entropy coding, we separate the quantized coefficients in two categories:  

*  the 'DC'='directe coefficient' coefficients are the coefficients (0,0) of each $8\times 8$-block.
* The 'AC' coefficients are the 63 coefficients remainings in each blocks (but actually, we trucate most of them).


The 'AC' ='alternative coefficient' coefficients are stored following the zigzag path, block by block, suppressing the zero sequency at the end of each block (see previous chapter).

The DC coefficients are the average values of the blocks. Because images are "continuous", the DC coefficients of two neighbor blocks are close. So the idea to store only the differences of the DC coefficients: we ordonnated the blocks, then we store DC(0) and  DiffDC(i) = DC(i) - DC(i-1). This achieve further compression due to the smaller range of the coefficient values.

The precise technic to encode of this DC and AC coefficients is still too long to explain. I send you to this [reference](https://en.wikibooks.org/wiki/JPEG_-_Idea_and_Practice/The_Huffman_coding) is you want to understand all details.


***To you:***

* $(1\heartsuit)$ Did you understant why "smaller range for the coefficient values" implies a better compression?
*  $(1\heartsuit)$ Look at the scheme above: What are the "Quantization tables"?
*  $(1\heartsuit)$ What are "Huffman table"? Why these tables appear at two different places in the scheme?


***Remark:*** Actually JPEG does not impose the Quantization tables and the Huffman table. When you create a JPEG file you can:

* either include tables
* or indicate that you want to use the default ones.


### Last words about  JPEG

We gave a lot of details about JPEG (not all) which is a family of standards edited from 1986. Of course it is not useful to know by heart all these: you will probably never have to write a JPEG compression program (even if you will probably become a chief ingenieur).  But this allow us to discover a lot of famous compression tricks:

    * change of basis
    * clever quantization
    * zigzag reading
    * color encoding
    * differential storing
    * huffman encoding
   
Secondly, JPEG is you a good example of collective intelligence, shared without patent! JPEG compression is so efficient that it became universal. More recent standards like JPEG2000 never manage to replace it.

The equivalent for video is MPEG (nowaday: version 4,  part 10). Of course it is not a compression frame by frame: it use the fact that a wide part of images does not evolute from a frame to another.  MPEG is also very efficient, but probably better standards will take the place in the futur (one speak a lot of the H.265 standard... wait an see).

Come back to JPEG: They are several way to precisely implement the compressing+decompressing process. In 2017, a team from google, playing with the quantization step, published the new implementation called "Guetzli":

* which reduce the file size of 35%
* which is fully compatible with the JPEG standards: so, no need to change the existing decoding codes (JPEG let you the choice of the quantizations table).
* but which require more calculus